In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, roc_curve

seed = 100
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# 氨基酸到索引的映射（1-20）
amino_to_index = {amino: i+1 for i, amino in enumerate('ARNDCQEGHILKMFPSTWYV')}

# 创建自定义数据集类
class ProteinDataset(Dataset):
    def __init__(self, data, amino_to_index, max_length=100):
        self.data = data
        self.amino_to_index = amino_to_index
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        # 将氨基酸序列转换为索引序列，并填充到指定的最大长度
        sequence_indices = [self.amino_to_index.get(amino, 0) for amino in sequence]
        sequence_indices += [0] * (self.max_length - len(sequence_indices))
        sequence_indices = sequence_indices[:self.max_length]

        return {
            'sequence_indices': torch.tensor(sequence_indices),
            'label': torch.tensor(label)
        }

# 从本地CSV文件中读取数据
data = pd.read_csv('At_Rice_all_1vs100.csv')

device = torch.device('cuda')
# Separating features (sequences) and labels
X = data.iloc[:, 0]  # The first column is the sequence
y = data.iloc[:, 1]  # The second column is the label

# Split the data ensuring consistent class ratios in train and test sets
train_X_1, test_X_1, train_y_1, test_y_1 = train_test_split(X[y == 1], y[y == 1], test_size=0.3, random_state=100)
train_X_0, test_X_0, train_y_0, test_y_0 = train_test_split(X[y == 0], y[y == 0], test_size=0.3, random_state=100)

train_X = pd.concat([train_X_1, train_X_0])
test_X = pd.concat([test_X_1, test_X_0])
train_y = pd.concat([train_y_1, train_y_0])
test_y = pd.concat([test_y_1, test_y_0])

# Combine features and labels for train and test sets and shuffle the data within each class to add randomness
train_data = shuffle(pd.DataFrame({'sequence': train_X, 'label': train_y}), random_state=100)
test_data = shuffle(pd.DataFrame({'sequence': test_X, 'label': test_y}), random_state=100)

# Create dataset objects for train and test sets
train_dataset = ProteinDataset(train_data, amino_to_index=amino_to_index, max_length=100)
test_dataset = ProteinDataset(test_data, amino_to_index=amino_to_index, max_length=100)

# Create data loaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 定义模型
class ProteinPredictor(nn.Module):
    def __init__(self, num_labels, max_length=100):
        super(ProteinPredictor, self).__init__()
        self.embedding = nn.Embedding(len(amino_to_index) + 1, 128)  # 加1用于填充
        self.transformer = nn.Transformer(d_model=128, nhead=8, num_encoder_layers=4)
        self.fc = nn.Linear(max_length * 128, num_labels)

    def forward(self, sequence_indices):
        embedded = self.embedding(sequence_indices)
        transformer_output = self.transformer(embedded.permute(1, 0, 2), embedded.permute(1, 0, 2))  # 使用源序列作为目标序列
        pooled_output = transformer_output.permute(1, 0, 2).reshape(sequence_indices.size(0), -1)
        logits = self.fc(pooled_output)
        return logits


num_labels = 2
model = ProteinPredictor(num_labels, max_length=100)
model.to(device)  # Move the model to GPU if available

# 定义损失函数和优化器，添加weight_decay参数
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

# 定义学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True) #ReduceLROnPlateau： 在验证集上的性能不再提升时降低学习率。


# Training and testing loops
num_epochs = 30
best_accuracy = 0.0
all_labels = []
all_scores = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
        sequence_indices = batch['sequence_indices'].to(device)  # Move data to GPU
        labels = batch['label'].to(device)  # Move data to GPU

        optimizer.zero_grad()
        outputs = model(sequence_indices)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)

    # Validation Loop
    model.eval()
    true_labels = []  # Initialize true_labels and predicted_labels lists
    predicted_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in test_dataloader:
            sequence_indices = batch['sequence_indices'].to(device)  # Move data to GPU
            labels = batch['label'].to(device)  # Move data to GPU

            outputs = model(sequence_indices)
            _, predicted = torch.max(outputs.data, 1)
            
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, 1]  # 获取预测为 positive 的概率

            all_labels.extend(labels.cpu().numpy())
            all_scores.extend(probabilities.cpu().numpy())

        auc = roc_auc_score(all_labels, all_scores)

        mcc = matthews_corrcoef(true_labels, predicted_labels)

        conf_matrix = confusion_matrix(true_labels, predicted_labels)


        # Precision指标衡量了模型在所有被预测为正例的样本中，有多少样本真正属于正例。
        #precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0

        tp = conf_matrix[1, 1]# 真阳性
        tn = conf_matrix[0, 0] # 真阴性
        fp = conf_matrix[0, 1] # 假阳性
        fn = conf_matrix[1, 0] # 假阴性

        sn = tp / (tp + fn) if (tp + fn) != 0 else 0.0 # 真阳性率（Sensitivity，也称为召回率或查全率）
        sp = tn / (tn + fp) if (tn + fp) != 0 else 0.0 # 特异性（Specificity）：表示被模型正确识别为负例的能力

        accuracy = (tp + tn) / (tp + tn + fp + fn)

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0 #Precision（精确率）是用于评估分类模型性能的指标之一。它衡量了在模型预测为正例的样本中，有多少是真正的正例
        f1_score = 2 * (precision * sn) / (precision + sn) if (precision + sn) != 0 else 0.0

        print('Epoch [{}/{}]\tAvg Loss: {:.4f}, True Positives: {:.4f}, True Negatives: {:.4f}, False Positives: {:.4f}, False Negatives: {:.4f},'
            .format(epoch+1, num_epochs, avg_loss, tp, tn, fp, fn))

        print('Epoch [{}/{}]\tTrain Loss: {:.4f}\tTest Accuracy: {:.2f}%, AUC: {:.4f}, Sn: {:.4f}, Sp: {:.4f}, Mcc: {:.4f}, Precision: {:.4f}, f1_score: {:.4f}'
            .format(epoch+1, num_epochs, avg_loss, accuracy * 100, auc, sn, sp, mcc, precision,f1_score))

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            # Save the model checkpoint
            torch.save(model.state_dict(), 'best_model_Trans.pth')
    # Update the learning rate scheduler
    scheduler.step(accuracy)


c:\Users\DELL\.conda\envs\pytorch_GPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch [1/30]	Avg Loss: 0.3615, True Positives: 3062.0000, True Negatives: 5277.0000, False Positives: 405.0000, False Negatives: 974.0000,
Epoch [1/30]	Train Loss: 0.3615	Test Accuracy: 85.81%, AUC: 0.9356, Sn: 0.7587, Sp: 0.9287, Mcc: 0.7071, Precision: 0.8832, f1_score: 0.8162
Epoch [2/30]	Avg Loss: 0.3173, True Positives: 3512.0000, True Negatives: 4879.0000, False Positives: 803.0000, False Negatives: 524.0000,
Epoch [2/30]	Train Loss: 0.3173	Test Accuracy: 86.34%, AUC: 0.9306, Sn: 0.8702, Sp: 0.8587, Mcc: 0.7229, Precision: 0.8139, f1_score: 0.8411
Epoch [3/30]	Avg Loss: 0.3007, True Positives: 2788.0000, True Negatives: 5479.0000, False Positives: 203.0000, False Negatives: 1248.0000,
Epoch [3/30]	Train Loss: 0.3007	Test Accuracy: 85.07%, AUC: 0.9316, Sn: 0.6908, Sp: 0.9643, Mcc: 0.6993, Precision: 0.9321, f1_score: 0.7935
Epoch [4/30]	Avg Loss: 0.2825, True Positives: 3120.0000, True Negatives: 5357.0000, False Positives: 325.0000, False Negatives: 916.0000,
Epoch [4/30]	Train L

In [2]:
# 保存模型
torch.save(model.state_dict(), 'protein_classifier.pth')
